In [1]:
import numpy as np
import pandas as pd

from IPython.core.display import display, HTML
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

import numpy as np
import pandas as pd
import pyarrow.parquet as pq
from fastai.tabular.all import df_shrink
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler

import seaborn as sns
from importlib import reload
import matplotlib.pyplot as plt
import matplotlib
import warnings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.expand_frame_repr', False)
# pd.set_option('max_colwidth', -1)
display(HTML("<style>div.output_scroll { height: 35em; }</style>"))

reload(plt)
%matplotlib inline
%config InlineBackend.figure_format ='retina'

warnings.filterwarnings('ignore')

# configure plotly graph objects
pio.renderers.default = 'iframe'
# pio.renderers.default = 'vscode'

pio.templates["ck_template"] = go.layout.Template(
    layout_colorway = px.colors.sequential.Viridis,
#     layout_hovermode = 'closest',
#     layout_hoverdistance = -1,
    layout_autosize=False,
    layout_width=800,
    layout_height=600,
    layout_font = dict(family="Calibri Light"),
    layout_title_font = dict(family="Calibri"),
    layout_hoverlabel_font = dict(family="Calibri Light"),
#     plot_bgcolor="white",
)

# pio.templates.default = 'seaborn+ck_template+gridon'
pio.templates.default = 'ck_template+gridon'
# pio.templates.default = 'seaborn+gridon'
# pio.templates

In [2]:
# https://www.kaggle.com/datasets/mrwellsdavid/unsw-nb15
train_df = pd.read_csv('/content/drive/MyDrive/UNSW_NB15/UNSW_NB15_training-set.csv')
test_df = pd.read_csv('/content/drive/MyDrive/UNSW_NB15/UNSW_NB15_testing-set.csv')


In [3]:
train_df.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,254,0,180363632.0,0.0,0,0,0.011,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,248,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,254,0,881000000.0,0.0,0,0,0.008,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,881,0,0,0,2,2,1,1,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,254,0,854400000.0,0.0,0,0,0.005,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,534,0,0,0,3,2,1,1,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,254,0,600000000.0,0.0,0,0,0.006,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,450,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,254,0,850400000.0,0.0,0,0,0.010,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,1063,0,0,0,3,2,2,2,1,3,0,0,0,2,3,0,Normal,0


In [4]:
list_drop = ['id','attack_cat']
train_df.drop(list_drop,axis=1,inplace=True)

In [5]:
DEBUG =0
train_df_numeric = train_df.select_dtypes(include=[np.number])

for feature in train_df_numeric.columns:
    if DEBUG == 1:
        print(feature)
        print('max = '+str(train_df_numeric[feature].max()))
        print('75th = '+str(train_df_numeric[feature].quantile(0.95)))
        print('median = '+str(train_df_numeric[feature].median()))
        print(train_df_numeric[feature].max()>10*train_df_numeric[feature].median())
        print('----------------------------------------------------')
    if train_df_numeric[feature].max()>10*train_df_numeric[feature].median() and train_df_numeric[feature].max()>10 :
        train_df[feature] = np.where(train_df[feature]<train_df[feature].quantile(0.95), train_df[feature], train_df[feature].quantile(0.95))

In [6]:
train_df_numeric = train_df.select_dtypes(include=[np.number])
train_df_before = train_df_numeric.copy()
DEBUG = 0
for feature in train_df_numeric.columns:
    if DEBUG == 1:
        print(feature)
        print('nunique = '+str(train_df_numeric[feature].nunique()))
        print(train_df_numeric[feature].nunique()>50)
        print('----------------------------------------------------')
    if train_df_numeric[feature].nunique()>50:
        if train_df_numeric[feature].min()==0:
            train_df[feature] = np.log(train_df[feature]+1)
        else:
            train_df[feature] = np.log(train_df[feature])

train_df_numeric = train_df.select_dtypes(include=[np.number])

In [7]:
train_df_cat = train_df.select_dtypes(exclude=[np.number])
train_df_cat.describe(include='all')

,proto,service,state
count,82332,82332,82332
unique,131,13,7
top,tcp,-,FIN
freq,43095,47153,39339


In [8]:
DEBUG = 0
for feature in train_df_cat.columns:
    if DEBUG == 1:
        print(feature)
        print('nunique = '+str(train_df_cat[feature].nunique()))
        print(train_df_cat[feature].nunique()>6)
        print(sum(train_df[feature].isin(train_df[feature].value_counts().head().index)))
        print('----------------------------------------------------')

    if train_df_cat[feature].nunique()>6:
        train_df[feature] = np.where(train_df[feature].isin(train_df[feature].value_counts().head().index), train_df[feature], '-')

In [9]:
train_df_cat = train_df.select_dtypes(exclude=[np.number])
train_df_cat.describe(include='all')

,proto,service,state
count,82332,82332,82332
unique,6,5,6
top,tcp,-,FIN
freq,43095,49275,39339


In [10]:
train_df['proto'].value_counts().head().index

Index(['tcp', 'udp', '-', 'unas', 'arp'], dtype='object', name='proto')

In [11]:
train_df['proto'].value_counts().index


Index(['tcp', 'udp', '-', 'unas', 'arp', 'ospf'], dtype='object', name='proto')

In [12]:
from typing import Union
import torch
import numpy as np


def normal_features(features:Union[list, np.ndarray, torch.Tensor], labels: Union[list, np.ndarray, torch.Tensor], noise: float = 0.4):
    if isinstance(labels, list):
        labels = np.array(labels)
    elif isinstance(labels, torch.Tensor):
        labels = labels.numpy()
    elif not isinstance(labels, np.ndarray):
        raise TypeError("The type of labels must be list, np.ndarray or torch.Tensor.")
    assert len(labels.shape) == 1, "The shape of labels must be (num_vertices, )."
    label_set = np.unique(labels).tolist()
    N, C = labels.shape[0], len(label_set)
    lebel_list = []
    for i in range(N):
        lebel_list.append(label_set.index(labels[i]))
    labels = np.array(lebel_list)
    centers = np.zeros((N, C))
    centers[np.arange(N), labels] = 1
    features_ = np.random.normal(centers, noise, size=(N, C))
    print(features_.shape)
    features[:, 2] = features_[:, 0]
    features[:, 3] = features_[:, 1]
    return features, labels


In [13]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
X = train_df.iloc[:,:-2]
X['proto'] = label_encoder.fit_transform(X['proto'])
X['service'] = label_encoder.fit_transform(X['service'])
X['state'] = label_encoder.fit_transform(X['state'])
y = train_df.iloc[:,-1]

In [18]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
def select_top_k_pearson(X, y, k=20):
    correlations = []
    for col in X.columns:
        corr, _ = pearsonr(X[col], y)
        correlations.append(abs(corr))

    top_k_indices = np.argsort(correlations)[-k:]
    return X.columns[top_k_indices]

def select_top_k_chi2(X, y, k=20):
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    chi2_selector = SelectKBest(chi2, k=k)
    chi2_selector.fit(X_scaled, y)
    return X.columns[chi2_selector.get_support()]

def select_top_k_anova(X, y, k=20):
    anova_selector = SelectKBest(f_classif, k=k)
    anova_selector.fit(X, y)
    return X.columns[anova_selector.get_support()]

def select_top_k_rfe(X, y, k=20):
    model = RandomForestClassifier(random_state=42)
    rfe_selector = RFE(estimator=model, n_features_to_select=k, step=1)
    rfe_selector.fit(X, y)
    return X.columns[rfe_selector.get_support()]

top_20_pearson = select_top_k_pearson(X, y, k=20)
top_20_chi2 = select_top_k_chi2(X, y, k=20)
top_20_anova = select_top_k_anova(X, y, k=20)
top_20_rfe = select_top_k_rfe(X, y, k=20)

# Combine the selected features
# selected_features = set(top_20_pearson).union(set(top_20_chi2)).union(set(top_20_anova))
# Only pearson features
selected_features = set(top_20_rfe)
final_selected_features = list(selected_features)[:20]  # If you want exactly 20 features

# Filter the dataset to keep only the selected features
X_selected = X[final_selected_features]

In [19]:
X_selected.shape, y.shape

((82332, 20), (82332,))

In [20]:
X, y = normal_features(X_selected.values, y.values)

(82332, 2)


In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=0,
                                                    stratify=y)


In [22]:
X_train

array([[ 6.52795792e+00,  1.51349829e+01,  7.18600879e-01, ...,
         2.19971843e+01,  7.61686544e+00,  2.55000000e+02],
       [ 5.36129217e+00,  1.17827116e+01,  9.89586838e-01, ...,
         2.13101993e+01,  5.13509487e+00,  2.55000000e+02],
       [ 0.00000000e+00,  0.00000000e+00, -2.10390563e-01, ...,
         0.00000000e+00,  1.27169012e+01,  0.00000000e+00],
       ...,
       [ 4.18965474e+00,  8.69312671e+00, -1.35604145e-01, ...,
         2.20275957e+01,  3.23177408e+00,  2.55000000e+02],
       [ 0.00000000e+00,  0.00000000e+00, -3.93133323e-01, ...,
         0.00000000e+00,  1.16182949e+01,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -5.72044130e-01, ...,
         0.00000000e+00,  1.18696074e+01,  0.00000000e+00]])

In [23]:
type(y_train)

numpy.ndarray

In [24]:
import torch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [25]:
X_test_tensor.shape

torch.Size([16467, 20])

In [26]:
X_train_tensor.shape

torch.Size([65865, 20])

In [27]:
!pip install lora-adapters peft

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

# TABNET with LoRA

In [28]:
pip install --upgrade peft

In [ ]:
from sparsemax import Sparsemax
import torch
from peft import LoraConfig

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam

# GLU Block
def glu(act, n_units):
    return act[:, :n_units] * torch.sigmoid(act[:, n_units:])

class LoRALinear(nn.Module):
    def __init__(self, in_features, out_features, rank, alpha=1):
        super(LoRALinear, self).__init__()
        self.rank = rank
        self.alpha = alpha

        # Original weight is frozen during training
        self.orig_weight = nn.Parameter(torch.Tensor(out_features, in_features), requires_grad=False)
        nn.init.kaiming_uniform_(self.orig_weight, a=math.sqrt(5))

        # Low-rank matrices U and V
        self.U = nn.Parameter(torch.Tensor(out_features, rank))
        self.V = nn.Parameter(torch.Tensor(rank, in_features))
        nn.init.kaiming_normal_(self.U)
        nn.init.kaiming_normal_(self.V)

    def forward(self, x):
        weight = self.orig_weight + self.alpha * self.U @ self.V
        return F.linear(x, weight)

    def extra_repr(self):
        return 'in_features={}, out_features={}, rank={}, alpha={}'.format(
            self.orig_weight.size(1), self.orig_weight.size(0), self.rank, self.alpha
        )

class TabNetModel(nn.Module):
    def __init__(
        self,
        num_features=56,
        feature_dims=56,
        output_dim=56,
        num_decision_steps=6,
        relaxation_factor=0.5,
        batch_momentum=0.001,
        virtual_batch_size=2,
        num_classes=2,
        epsilon=0.00001
    ):
        super().__init__()

        self.num_features = num_features
        self.feature_dims = feature_dims
        self.output_dim = output_dim
        self.num_decision_steps = num_decision_steps
        self.relaxation_factor = relaxation_factor
        self.batch_momentum = batch_momentum
        self.virtual_batch_size = virtual_batch_size
        self.num_classes = num_classes
        self.epsilon = epsilon

        self.feature_transform_linear1 = LoRALinear(num_features, self.feature_dims * 2, rank=16)
        self.BN = torch.nn.BatchNorm1d(num_features, momentum=batch_momentum)
        self.BN1 = torch.nn.BatchNorm1d(self.feature_dims * 2, momentum=batch_momentum)

        self.feature_transform_linear2 = torch.nn.Linear(self.feature_dims * 2, self.feature_dims * 2, bias=False)
        self.feature_transform_linear3 = torch.nn.Linear(self.feature_dims, self.feature_dims * 2, bias=False)
        self.feature_transform_linear4 = torch.nn.Linear(self.feature_dims * 2, self.feature_dims * 2, bias=False)

        self.mask_linear_layer = torch.nn.Linear(self.feature_dims * 2 - output_dim, self.num_features, bias=False)
        self.BN2 = torch.nn.BatchNorm1d(self.num_features, momentum=batch_momentum)

        self.final_classifier_layer = torch.nn.Linear(self.output_dim, self.num_classes, bias=False)
        self.sparsemax = nn.Softmax(dim=1)  # Changed to nn.Softmax for compatibility

    def encoder(self, data):
        batch_size = data.shape[0]
        features = self.BN(data)
        output_aggregated = torch.zeros([batch_size, self.output_dim], dtype=torch.float).to(device)

        masked_features = features
        mask_values = torch.zeros([batch_size, self.num_features]).to(device)

        aggregated_mask_values = torch.zeros([batch_size, self.num_features]).to(device)
        complemantary_aggregated_mask_values = torch.ones([batch_size, self.num_features]).to(device)

        total_entropy = 0

        for ni in range(self.num_decision_steps):
            if ni == 0:
                transform_f1 = self.feature_transform_linear1(masked_features)
                norm_transform_f1 = self.BN1(transform_f1)
                transform_f2 = self.feature_transform_linear2(norm_transform_f1)
                norm_transform_f2 = self.BN1(transform_f2)
            else:
                transform_f1 = self.feature_transform_linear1(masked_features)
                norm_transform_f1 = self.BN1(transform_f1)
                transform_f2 = self.feature_transform_linear2(norm_transform_f1)
                norm_transform_f2 = self.BN1(transform_f2)
                transform_f2 = (glu(norm_transform_f2, self.feature_dims) + transform_f1[:, :self.feature_dims]) * np.sqrt(0.5)
                transform_f3 = self.feature_transform_linear3(transform_f2)
                norm_transform_f3 = self.BN1(transform_f3)
                transform_f4 = self.feature_transform_linear4(norm_transform_f3)
                norm_transform_f4 = self.BN1(transform_f4)
                transform_f4 = (glu(norm_transform_f4, self.feature_dims) + transform_f3[:, :self.feature_dims]) * np.sqrt(0.5)
                decision_out = torch.nn.ReLU(inplace=True)(transform_f4[:, :self.output_dim])
                output_aggregated = torch.add(decision_out, output_aggregated)
                scale_agg = torch.sum(decision_out, axis=1, keepdim=True) / (self.num_decision_steps - 1)
                aggregated_mask_values = torch.add(aggregated_mask_values, mask_values * scale_agg)
                features_for_coef = transform_f4[:, :]
                if ni < (self.num_decision_steps - 1):
                    mask_linear_layer = self.mask_linear_layer(features_for_coef)
                    mask_linear_norm = self.BN2(mask_linear_layer)
                    mask_linear_norm = torch.mul(mask_linear_norm, complemantary_aggregated_mask_values)
                    mask_values = self.sparsemax(mask_linear_norm)
                    complemantary_aggregated_mask_values = torch.mul(complemantary_aggregated_mask_values, self.relaxation_factor - mask_values)
                    total_entropy = torch.add(total_entropy, torch.mean(torch.sum(-mask_values * torch.log(mask_values + self.epsilon), axis=1)) / (self.num_decision_steps - 1))
                    masked_features = torch.mul(mask_values, features)

        return output_aggregated, total_entropy

    def classify(self, output_logits):
        logits = self.final_classifier_layer(output_logits)
        predictions = F.softmax(logits, dim=1)
        return logits, predictions




In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [32]:
# Data Preparation
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

num_features = X_train_tensor.shape[1]
model = TabNetModel(num_features=num_features, num_classes=2)
model.load_state_dict(torch.load("/content/drive/MyDrive/TabLoRA_pretrained_models/tabnet_bot_iot_v2_anova.pt"))
model = model.to(device)

for name, param in model.named_parameters():
    if 'U' in name or 'V' in name:
        param.requires_grad = False
    else:
        param.requires_grad = True

optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
loss_function = torch.nn.CrossEntropyLoss()

def train_model(model, train_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            optimizer.zero_grad()
            output, _ = model.encoder(X_batch)
            loss = loss_function(model.classify(output)[0], y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

train_model(model, train_loader)

Epoch 1, Loss: 0.07490693795883539
Epoch 2, Loss: 0.053672367702147845
Epoch 3, Loss: 0.050336409529368166
Epoch 4, Loss: 0.04772070394958894
Epoch 5, Loss: 0.046930171214541405
Epoch 6, Loss: 0.04516293742960699
Epoch 7, Loss: 0.04468763255145744
Epoch 8, Loss: 0.04305600683053377
Epoch 9, Loss: 0.04095639204188223
Epoch 10, Loss: 0.03998966579821675


In [33]:

from sklearn.metrics import accuracy_score, f1_score, precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt

def evaluate_model(model, X_test_tensor, y_test_tensor):
  model.eval()
  with torch.no_grad():
    X_test_batch, y_test_batch = normal_features(X_test_tensor.numpy(), y_test_tensor.numpy())
    X_test_batch = torch.tensor(X_test_batch)
    y_test_batch = torch.tensor(y_test_batch)
    X_test_batch, y_test_batch = X_test_batch.to(device), y_test_batch.to(device)
    output, _ = model.encoder(X_test_batch)
    logits, predictions = model.classify(output)
    print(logits.shape, predictions.shape)
    accuracy = accuracy_score(y_test_batch.cpu(), logits.argmax(dim=1).cpu())
    f1 = f1_score(y_test_batch.cpu(), predictions.argmax(dim=1).cpu())
    precision, recall, _ = precision_recall_curve(y_test_batch.cpu(), predictions[:, 1].cpu())
    average_precision = average_precision_score(y_test_batch.cpu(), predictions[:, 1].cpu())

    print("Accuracy:", accuracy)
    print("F1 Score:", f1)
    print("Average Precision:", average_precision)

  return accuracy, f1, average_precision

accuracy, f1, average_precision = evaluate_model(model, X_test_tensor, y_test_tensor)

(16467, 2)
torch.Size([16467, 2]) torch.Size([16467, 2])
Accuracy: 0.9740086233072205
F1 Score: 0.9762169370971328
Average Precision: 0.9978713258488026


In [34]:
torch.save(model.state_dict(), "/content/drive/MyDrive/TabLoRA_pretrained_models/tabnet_unsw_v2_rfe.pt")